In [ ]:
#Connect to PowerBI Dataset within Python Notebook

#Tutorial from
# https://pawarbi.github.io/blog/ppu/xmla/powerbi_premium/premium/python/jupyter_notebook/2020/12/11/Accessing-Power-BI-Datasets-via-XMLA-Endpoint-in-Python-Jupyter-Notebook.html


In [ ]:
pip install pythonnet

In [ ]:
import pandas as pd
import ssas_api as ssas
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# server is found under Workspace settings within Power BI Online under Premium
# copy under Workspace Connection

server = '' #find link on Power BI online workspace
username = '' #your email address
password = '' # make sure to clear your password

In [ ]:
# must add this file into your active repo
# https://github.com/yehoshuadimarsky/python-ssas/blob/master/ssas_api.py

conn1 = ssas.set_conn_string(
    server=server,
    db_name='',
    username=username,
    password=password
    )

In [ ]:
global System, DataTable, AMO, ADOMD

import System
from System.Data import DataTable
import Microsoft.AnalysisServices.Tabular as TOM
import Microsoft.AnalysisServices.AdomdClient as ADOMD

try:
    TOMServer = TOM.Server()
    TOMServer.Connect(conn1)
    print("Connection to Workspace Successful !")
    
except:
    print("Connection to Workspace Failed")

In [ ]:
pd.set_option('display.max_rows', None) # print full list
datasets = pd.DataFrame(columns=['Dataset_Name', 'Compatibility', 'ID', 'Size_MB','Created_Date','Last_Update' ])

for item in TOMServer.Databases:
    
          
    datasets = datasets.append({'Dataset_Name' :item.Name, 
                     'Compatibility':item.CompatibilityLevel,
                     'Created_Date' :item.CreatedTimestamp,
                     'ID'           :item.ID,
                     'Last_Update'  :item.LastUpdate,
                     'Size_MB'      :(item.EstimatedSize*1e-06)    },
                     ignore_index=True)
    
datasets

In [ ]:
#choose dataset Id
#show all tables within dataset

ds = TOMServer.Databases[''] # add id from desired dataset in above output

for table in ds.Model.Tables:
    print(table.Name)

In [ ]:
#connect to table within above dataset using Dataset_name

conn2 = (ssas.set_conn_string(
    server=server,
    db_name='',
    username = username,
    password = password
 ))

In [ ]:
# You can write any valid DAX query against this table and get the results back as a Pandas dataframe.

dax_string = '''
    //Write your DAX Query here
    EVALUATE
    //TableName
    '''

df = (ssas
      .get_DAX(
      connection_string=conn2, 
      dax_string=dax_string)         
              )

df.head(10)

In [ ]:
for col in df.columns:
    print(col)

#totalReplacements=df.sum(Replacements)
#totalReplacements

In [ ]:
# change column names
df = df.rename(columns=lambda x: x.split("[")[1])
df.columns = df.columns.str.replace("]", "")
# print dataframe columns
print("Dataframe columns:", df.columns)

In [ ]:
#show column names

for col in df.columns:
    print(col)

In [ ]:
dataTypes = df.dtypes
dataTypes

In [ ]:
df.Replacements = df.Replacements.astype(int)
df['Date'] = pd.to_datetime(df['Created']).dt.date

In [ ]:
totalReplacements = sum(df.Replacements)
totalReplacements

In [ ]:
tmp = df.groupby('Date')['Replacements'].sum().reset_index()
tmp['Replacements_cumsum'] = tmp['Replacements'].cumsum()

In [ ]:
print(tmp.head(10))

print("Average Replacements Per Day " + str(round(tmp["Replacements"].mean(), 2)))
print("Max Replacements Per Day " + str(tmp["Replacements"].max()))

In [ ]:
plt.xticks(rotation=30)
plt.plot(tmp.Date, tmp.Replacements_cumsum)